In [1]:
import wandb
import pandas as pd
import glob
import os
import yaml
import numpy as np

In [2]:
#Directory with .csv and .yaml files
root_dir = r"C:\Users\lfcas\Documents\Thesis Paper\ma_casas\results\9x3x2\9x3x2_ins_0"

In [3]:
# Load .yaml
config =  yaml.load(open(os.path.join(root_dir, "configuration.yaml")), Loader=yaml.Loader)

# Initialize Run
run = wandb.init(
    project = "ma_casas",
    tags = ["9x3x2", "Instance 0"],
    name= "9x3x2_ins_0",
    config = config,
    settings=wandb.Settings(_disable_stats=True),
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
C:\Users\lfcas\.conda\envs\unity_rl\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: Currently logged in as: huataran (ma_casas). Use `wandb login --relogin` to force relogin


In [ ]:
# Load .csv files
all_files = glob.glob(os.path.join(root_dir, "*.csv")) 
dfs = list()
df = pd.read_csv(all_files[0])
df = df.rename(columns ={"Value": all_files[0].split("tag-")[-1][:-4]})

for f in all_files[1:]:
    data = pd.read_csv(f)
    data = data.rename(columns={"Value": f.split("tag-")[-1][:-4]})
    del data ["Wall time"]
    df = df.merge(data, how="left", on="Step")


# Log Data
df.reset_index()
keys = df.keys()

for index, row in df.iterrows():
    for key in keys:
        if(not np.isnan(row[key])):
            run.log({key: row[key]},step = int(row["Step"]))

df.reset_index()

# Log Table with original data
table = wandb.Table(dataframe = df)
run.log({"Original Data": table})

    

In [ ]:
run.finish()